In [4]:
import pandas as pd
import numpy as np

# 1. Veriyi pkl dosyasından yükle
file_path = 'birlesik_veri_tekil.pkl'
data = pd.read_pickle(file_path)

# İlk birkaç satırı görüntüle
print("Veri Setindeki İlk 5 Satır:")
print(data.head())

print("\n")


Veri Setindeki İlk 5 Satır:
       uye_id  urun_id           eklenme_tarihi  puan   event_type  \
0           0    73914  2024-08-05 06:04:45.000     3  add_to_cart   
26803       0    67497  2024-08-05 06:04:45.000     3  add_to_cart   
51084       0    71618  2024-08-05 07:40:59.000     2         view   
51456       0    72386  2024-08-05 07:41:00.000     4  add_to_cart   
51554       0    47037  2024-08-05 07:41:01.000     5     purchase   

                                                     adi marka_id kategori_id  \
0      Electrolux  SuperCyclone SCPARKETTO 1400W Toz ...      502        1366   
26803  Logitech MX Keys 920-010087 TR Q Aydınlatmalı ...       67         746   
51084  Toshiba N300 HDWG21EUZSVA 14TB 7200RPM 256MB 3...      120         794   
51456  Asrock X870 Pro RS WiFi AMD X870 Soket AM5 DDR...       15         779   
51554  Asus ROG Strix LC 240 RGB White Edition 240mm ...       16        1775   

                                             ozellikleri  
0    

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Content-Based Filtering (İçerik Tabanlı)
def content_based(data, urun_id, top_n=5):
    # TF-IDF vektörleştirme
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(data['ozellikleri'])
    
    # Cosine Similarity hesaplama
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Öneri
    try:
        idx = data[data['urun_id'] == urun_id].index[0]
        benzerlik_skorları = list(enumerate(cosine_sim[idx]))
        benzerlik_skorları = sorted(benzerlik_skorları, key=lambda x: x[1], reverse=True)
        benzer_urunler = [data.iloc[i[0]]['urun_id'] for i in benzerlik_skorları[1:top_n+1]]  # İlk top_n öneri
        return benzer_urunler
    except IndexError:
        return []

# Collaborative Filtering (İşbirlikçi)
def collaborative_filtering(user_item_matrix, urun_id, top_n=5):
    # Cosine Similarity ile ürün benzerlikleri
    cosine_sim = cosine_similarity(user_item_matrix.T, user_item_matrix.T)
    product_idx = user_item_matrix.columns.get_loc(urun_id) if urun_id in user_item_matrix.columns else None
    
    if product_idx is not None:
        benzerlik_skorları = list(enumerate(cosine_sim[product_idx]))
        benzerlik_skorları = sorted(benzerlik_skorları, key=lambda x: x[1], reverse=True)
        benzer_urunler = [user_item_matrix.columns[i[0]] for i in benzerlik_skorları[1:top_n+1]]  # İlk top_n öneri
        return benzer_urunler
    else:
        return []

# Hybrid Öneri Sistemi
# Hybrid öneri fonksiyonu güncellemesi
def hybrid_recommendation(data, user_item_matrix, urun_id, top_n=10):
    # Ürün ID'nin mevcut olup olmadığını kontrol et
    if urun_id not in data['urun_id'].values:
        return f"Ürün ID {urun_id} veri setinde bulunamadı."
    
    # Content-Based ve Collaborative Filtering önerileri
    content_based_scores = content_based(data, urun_id, top_n=top_n)
    collaborative_scores = collaborative_filtering(user_item_matrix, urun_id, top_n=top_n)
    
    # Her iki sonuç birleşimi
    combined_scores = list(set(content_based_scores + collaborative_scores))
    
    # Hiç öneri yoksa mesaj döndür
    if not combined_scores:
        return f"Ürün ID {urun_id} için öneri bulunamadı."
    
    # İlk top_n öneriyi döndür
    final_recommendations = combined_scores[:top_n]
    return final_recommendations

user_item_matrix = data.pivot_table(index='uye_id', columns='urun_id', values='puan', fill_value=0)

# Rastgele bir ürün ID'si dene
rastgele_urun_id = 73914  # Rastgele ürün ID
önerilen_urunler = hybrid_recommendation(data, user_item_matrix, rastgele_urun_id)

# Önerileri yazdır
if isinstance(önerilen_urunler, str):  # Hata mesajı dönerse
    print(önerilen_urunler)
else:
    önerilen_urun_detaylari = data[data['urun_id'].isin(önerilen_urunler)].drop_duplicates(subset=['urun_id'])
    print(f"Önerilen ürünlerin detayları:\n{önerilen_urun_detaylari}")


Önerilen ürünlerin detayları:
       uye_id  urun_id           eklenme_tarihi  puan   event_type  \
67713       0    26543  2024-08-06 14:05:16.000     1         view   
68012       0    66189  2024-08-06 15:17:58.000     5     purchase   
70754       0    50073  2024-08-07 14:41:18.000     2         view   
74295       0    67221  2024-08-11 19:31:33.000     4  add_to_cart   
74392       0    29211  2024-08-12 09:36:58.000     2         view   
75821       0    26529  2024-08-15 02:09:35.000     2         view   
78617       0    13827  2024-08-21 15:34:03.000     3  add_to_cart   
79652       0    25110  2024-08-23 16:02:32.000     1         view   
80992  433507    32274  2024-09-06 16:09:52.000     2         view   
81107       0    18211  2024-09-12 19:49:12.000     2         view   

                                                     adi marka_id kategori_id  \
67713  MSI Summit E16 Flip A11UCT-011TR i7-1195G7 16G...       75         834   
68012  Fakir Veyron Turbo XL Rose 750

In [15]:
user_id = 0  # Kullanıcı ID'si (örnek)
relevant_items = data[data['uye_id'] == user_id]['urun_id'].tolist()  # Kullanıcı 1 için etkileşimde bulunduğu ürünler

In [17]:
def calculate_mns(önerilen_urunler, item_popularity):
    novelty_scores = [1 / (1 + item_popularity[item]) for item in önerilen_urunler if item in item_popularity]
    return sum(novelty_scores) / len(novelty_scores) if novelty_scores else 0

item_popularity = data['urun_id'].value_counts().to_dict()
mns_score = calculate_mns(önerilen_urunler, item_popularity)
print(f"Mean Novelty Score (MNS): {mns_score:.4f}")


Mean Novelty Score (MNS): 0.5000


In [19]:
def calculate_mds(önerilen_urunler, item_categories):
    categories = [item_categories[item] for item in önerilen_urunler if item in item_categories]
    return len(set(categories)) / len(categories) if categories else 0

item_categories = data.set_index('urun_id')['kategori_id'].to_dict()
mds_score = calculate_mds(önerilen_urunler, item_categories)
print(f"Mean Diversity Score (MDS): {mds_score:.4f}")

Mean Diversity Score (MDS): 0.8000


In [20]:
def calculate_cc(önerilen_urunler, total_items):
    return len(set(önerilen_urunler)) / len(total_items)

total_items = data['urun_id'].unique()
cc_score = calculate_cc(önerilen_urunler, total_items)
print(f"Catalog Coverage (CC): {cc_score:.4f}")

Catalog Coverage (CC): 0.0112


In [21]:
def calculate_map(önerilen_urunler, relevant_items):
    precision_scores = []
    for k in range(1, len(önerilen_urunler) + 1):
        precision_at_k = len(set(önerilen_urunler[:k]) & set(relevant_items)) / k
        precision_scores.append(precision_at_k)
        
    return sum(precision_scores) / len(relevant_items) if relevant_items else 0

map_score = calculate_map(önerilen_urunler, relevant_items)
print(f"MAP Score: {map_score:.4f}")

MAP Score: 0.0137


In [22]:
def calculate_mps(data, user_item_matrix, top_n=5):
    # Kullanıcıların her birine öneriler oluştur
    user_recommendations = {}
    
    for user_id in data['uye_id'].unique():
        # Kullanıcı için öneri al
        # Önerilen ürünler, hybrid_recommendation fonksiyonu ile dinamik alınabilir
        recommended_items = hybrid_recommendation(data, user_item_matrix, user_id, top_n=top_n)
        user_recommendations[user_id] = recommended_items
    
    # MPS Hesaplama
    total_users = len(user_recommendations)
    unique_recommendations = sum([len(set(user_recommendations[u])) for u in user_recommendations])
    total_recommendations = sum([len(user_recommendations[u]) for u in user_recommendations])
    
    # MPS (Mean Personalization Score) hesapla
    return unique_recommendations / total_recommendations if total_recommendations > 0 else 0

# Kullanım:
mps_score = calculate_mps(data, user_item_matrix, top_n=5)
print(f"Mean Personalization Score (MPS): {mps_score:.4f}")


Mean Personalization Score (MPS): 0.6304


In [23]:
import pickle

# Tüm metrikleri tek bir sözlükte topla
hybrid_results = {
    "MAP": map_score,
    "CC": cc_score,
    "MNS": mns_score,
    "MDS": mds_score,
    "MPS": mps_score
}

# Tek bir dosyaya kaydet
with open("hybrid_sonuclari.pkl", "wb") as f:
    pickle.dump(hybrid_results, f)
